In [ ]:
!pip install pycryptodome
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip install torch
!pip -q install sentence_transformers
!pip install accelerate>=0.21.0
!pip install transformers
!pip install pandas
!pip install torch
!pip install scikit-learn
!pip install pycryptodome
!pip install gdown
!pip install nltk
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import requests
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
import requests
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
import requests
import nltk
import random
from nltk.util import ngrams
from collections import defaultdict, Counter
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
from sklearn.model_selection import train_test_split
import requests
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP



In [ ]:
!pip install torch==2.2.1

!pip install --upgrade fastai torchaudio torchtext torchvision

# training

In [ ]:
import pandas as pd
import gdown
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
import os
from google.colab import drive

# Function to load dataset from file
def load_dataset_from_file(file_path):
    df = pd.read_csv(file_path)
    return df

# Google Drive file IDs
scenario1_file_id = "1JEVXUblvt7gd0OIgT84SjTU30VyMi67V"
scenario2_file_id = "16g6vuDCkPUUGmjHWi9F9KDPaxuclW2ox"

# Download files from Google Drive using file IDs
scenario1_file_url = f"https://drive.google.com/uc?id={scenario1_file_id}"
scenario2_file_url = f"https://drive.google.com/uc?id={scenario2_file_id}"

scenario1_file_path = "/content/scenario1.csv"
scenario2_file_path = "/content/scenario2.csv"

gdown.download(scenario1_file_url, scenario1_file_path, quiet=False)
gdown.download(scenario2_file_url, scenario2_file_path, quiet=False)

# Load datasets from local files
scenario1_df = load_dataset_from_file(scenario1_file_path)
scenario2_df = load_dataset_from_file(scenario2_file_path)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer with the specified pad token ID
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token=str(50257))

# Tokenize the input text for scenario 1
train_encodings_scenario1 = tokenizer(scenario1_df['Drug_Name1'].tolist(), scenario1_df['Drug_Name2'].tolist(), truncation=True, padding=True)
train_labels_scenario1 = torch.tensor(scenario1_df['DDI_Label'].tolist())
# Create PyTorch dataset for scenario 1
train_dataset_scenario1 = TensorDataset(torch.tensor(train_encodings_scenario1['input_ids']),
                                        torch.tensor(train_encodings_scenario1['attention_mask']),
                                        train_labels_scenario1)
# Define data loader for scenario 1
batch_size = 16  # adjust batch size as needed
train_dataloader_scenario1 = DataLoader(train_dataset_scenario1, batch_size=batch_size, shuffle=True)

# Tokenize the input text for scenario 2
# Assuming 'Patient_Name' and 'Drug_Name' are columns in scenario2_df
# Concatenate 'Patient_Name' and 'Drug_Name' for input
input_text_scenario2 = scenario2_df['Patient_Name'] + " " + scenario2_df['Drug_Name']
train_encodings_scenario2 = tokenizer(input_text_scenario2.tolist(), truncation=True, padding=True)
train_labels_scenario2 = torch.tensor(scenario2_df['DDI_Label'].tolist())
# Create PyTorch dataset for scenario 2
train_dataset_scenario2 = TensorDataset(torch.tensor(train_encodings_scenario2['input_ids']),
                                        torch.tensor(train_encodings_scenario2['attention_mask']),
                                        train_labels_scenario2)
# Define data loader for scenario 2
train_dataloader_scenario2 = DataLoader(train_dataset_scenario2, batch_size=batch_size, shuffle=True)

# Load the pre-trained GPT2 model for sequence classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.resize_token_embeddings(len(tokenizer))  # Resize token embeddings to match the tokenizer's vocabulary size
model.to(device)  # Move the model to the appropriate device

# Ensure that the model's configuration has the pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader_scenario1))

# Training loop for scenario 1
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader_scenario1:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional: gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader_scenario1)
    print(f'Scenario 1 - Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}')

# Training loop for scenario 2
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader_scenario2:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional: gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader_scenario2)
    print(f'Scenario 2 - Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}')

# Specify the directory path where the notebook file is located
notebook_directory = '/content/drive/My Drive/Colab Notebooks'

# Save the model to the notebook directory
save_path = os.path.join(notebook_directory, "Lastmodel.pth")
torch.save(model.state_dict(), save_path)
print("Model saved successfully!" if os.path.exists(save_path) else "Failed to save the model.")


Downloading...
From: https://drive.google.com/uc?id=1JEVXUblvt7gd0OIgT84SjTU30VyMi67V
To: /content/scenario1.csv
100%|██████████| 1.20M/1.20M [00:00<00:00, 35.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=16g6vuDCkPUUGmjHWi9F9KDPaxuclW2ox
To: /content/scenario2.csv
100%|██████████| 1.38M/1.38M [00:00<00:00, 40.1MB/s]
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Scenario 1 - Epoch 1/3, Training Loss: 0.7147
Scenario 1 - Epoch 2/3, Training Loss: 0.6911
Scenario 1 - Epoch 3/3, Training Loss: 0.6883
Scenario 2 - Epoch 1/3, Training Loss: 0.6909
Scenario 2 - Epoch 2/3, Training Loss: 0.6907
Scenario 2 - Epoch 3/3, Training Loss: 0.6936
Model saved successfully!


In [ ]:
import pandas as pd
import gdown
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer  # Import necessary modules
import os
from google.colab import drive

# Function to load dataset from file
def load_dataset_from_file(file_path):
    df = pd.read_csv(file_path)
    return df

# Google Drive file IDs
scenario1_file_id = "1JEVXUblvt7gd0OIgT84SjTU30VyMi67V"
scenario2_file_id = "16g6vuDCkPUUGmjHWi9F9KDPaxuclW2ox"

# Download files from Google Drive using file IDs
scenario1_file_url = f"https://drive.google.com/uc?id={scenario1_file_id}"
scenario2_file_url = f"https://drive.google.com/uc?id={scenario2_file_id}"

scenario1_file_path = "/content/scenario1.csv"
scenario2_file_path = "/content/scenario2.csv"

gdown.download(scenario1_file_url, scenario1_file_path, quiet=False)
gdown.download(scenario2_file_url, scenario2_file_path, quiet=False)

# Load datasets from local files
scenario1_df = load_dataset_from_file(scenario1_file_path)
scenario2_df = load_dataset_from_file(scenario2_file_path)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer with the specified pad token ID
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token=str(50257))

# Tokenize the input text for scenario 1
train_encodings_scenario1 = tokenizer(scenario1_df['Drug_Name1'].tolist(), scenario1_df['Drug_Name2'].tolist(), truncation=True, padding=True)
train_labels_scenario1 = torch.tensor(scenario1_df['DDI_Label'].tolist())
# Create PyTorch dataset for scenario 1
train_dataset_scenario1 = TensorDataset(torch.tensor(train_encodings_scenario1['input_ids']),
                                        torch.tensor(train_encodings_scenario1['attention_mask']),
                                        train_labels_scenario1)
# Define data loader for scenario 1
batch_size = 16  # adjust batch size as needed
train_dataloader_scenario1 = DataLoader(train_dataset_scenario1, batch_size=batch_size, shuffle=True)

# Tokenize the input text for scenario 2
# Assuming 'Patient_Name' and 'Drug_Name' are columns in scenario2_df
# Concatenate 'Patient_Name' and 'Drug_Name' for input
input_text_scenario2 = scenario2_df['Patient_Name'] + " " + scenario2_df['Drug_Name']
train_encodings_scenario2 = tokenizer(input_text_scenario2.tolist(), truncation=True, padding=True)
train_labels_scenario2 = torch.tensor(scenario2_df['DDI_Label'].tolist())
# Create PyTorch dataset for scenario 2
train_dataset_scenario2 = TensorDataset(torch.tensor(train_encodings_scenario2['input_ids']),
                                        torch.tensor(train_encodings_scenario2['attention_mask']),
                                        train_labels_scenario2)
# Define data loader for scenario 2
train_dataloader_scenario2 = DataLoader(train_dataset_scenario2, batch_size=batch_size, shuffle=True)

# Load the pre-trained GPT2 model for sequence classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.resize_token_embeddings(len(tokenizer))  # Resize token embeddings to match the tokenizer's vocabulary size
model.to(device)  # Move the model to the appropriate device

# Ensure that the model's configuration has the pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader_scenario1))

# Training loop for scenario 1
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader_scenario1:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional: gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader_scenario1)
    print(f'Scenario 1 - Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}')

# Training loop for scenario 2
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader_scenario2:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional: gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader_scenario2)
    print(f'Scenario 2 - Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}')

# Specify the directory path where the notebook file is located
notebook_directory = '/content/drive/My Drive/Colab Notebooks'

# Save the model to the notebook directory
save_path = os.path.join(notebook_directory, "Lastmodel2.pth")
torch.save(model.state_dict(), save_path)
print("Model saved successfully!" if os.path.exists(save_path) else "Failed to save the model.")


Downloading...
From: https://drive.google.com/uc?id=1JEVXUblvt7gd0OIgT84SjTU30VyMi67V
To: /content/scenario1.csv
100%|██████████| 1.20M/1.20M [00:00<00:00, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=16g6vuDCkPUUGmjHWi9F9KDPaxuclW2ox
To: /content/scenario2.csv
100%|██████████| 1.38M/1.38M [00:00<00:00, 140MB/s]


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Scenario 1 - Epoch 1/3, Training Loss: 0.7397
Scenario 1 - Epoch 2/3, Training Loss: 0.6950
Scenario 1 - Epoch 3/3, Training Loss: 0.6937
Scenario 2 - Epoch 1/3, Training Loss: 0.6949
Scenario 2 - Epoch 2/3, Training Loss: 0.6938
Scenario 2 - Epoch 3/3, Training Loss: 0.6945
Model saved successfully!


Model Size (bytes): 497814230 Tokenizer vocabulary size: 5025

In [ ]:
from transformers import GPT2ForSequenceClassification, GPT2Config
import torch

# Path to the pre-trained model weights for the combined model
combined_model_weights_path = "/content/drive/My Drive/Colab Notebooks/model22L.pth"

# Load the model architecture with the default configuration
model = GPT2ForSequenceClassification.from_pretrained('gpt2')

# Load the state dictionary
state_dict = torch.load(combined_model_weights_path, map_location=torch.device('cpu'))

# Check if the model is a GPT2ForSequenceClassification
if not isinstance(model, GPT2ForSequenceClassification):
    raise TypeError("Model must be of type GPT2ForSequenceClassification.")

# Check if the model has the 'transformer' attribute
if not hasattr(model, 'transformer'):
    raise AttributeError("Model does not have a 'transformer' attribute.")

# Resize the embedding layer if needed
if state_dict['transformer.wte.weight'].shape[0] != model.transformer.wte.weight.shape[0]:
    model.resize_token_embeddings(state_dict['transformer.wte.weight'].shape[0])

# Load the state dictionary
model.load_state_dict(state_dict, strict=False)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

# evalution

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import requests
import io

# Placeholder function for loading your trained model
def load_model(model_path):
    # Load the pretrained GPT2 model for sequence classification
    model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
    model.resize_token_embeddings(len(tokenizer))  # Resize token embeddings to match the tokenizer's vocabulary size

    # Set the pad token ID
    model.config.pad_token_id = tokenizer.pad_token_id

    # Load the trained model state dictionary
    model_state_dict = torch.load(model_path, map_location=torch.device('cpu'))

    # Get the model's configuration
    model_config = model.config

    # Check if the model's vocabulary size matches the vocabulary size in the checkpoint
    if model_config.vocab_size != model_state_dict['transformer.wte.weight'].size(0):
        print("Resizing the embedding layer...")
        # Resize the embedding layer to match the vocabulary size
        model.resize_token_embeddings(model_state_dict['transformer.wte.weight'].size(0))
        print("Embedding layer resized.")

    # Load the model state dictionary
    model.load_state_dict(model_state_dict, strict=False)

    return model

# Function to load dataset from URL
def load_dataset_from_url(url):
    response = requests.get(url)
    df = pd.read_csv(io.StringIO(response.text))
    return df

# Function to evaluate the model
def evaluate_model(model, tokenizer, test_data, scenario):
    # Tokenize the input text
    if scenario == 1:
        test_encodings = tokenizer(test_data['Drug_Name1'].tolist(), test_data['Drug_Name2'].tolist(), truncation=True, padding=True)
    elif scenario == 2:
        test_encodings = tokenizer(test_data['Patient_Name'].tolist(), test_data['Drug_Name'].tolist(), truncation=True, padding=True)

    # Create PyTorch dataset for test data
    test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                                 torch.tensor(test_encodings['attention_mask']),
                                 torch.tensor(test_data['DDI_Label'].tolist()))

    # Define data loader for test data
    batch_size = 16  # adjust batch size as needed
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    # Lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over test data batches
    for batch in test_dataloader:
        # Move batch to the same device as the model
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Get predicted labels
        predicted_class = torch.argmax(logits, dim=-1)

        # Append true and predicted labels
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predicted_class.tolist())

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, target_names=['No Interaction', 'Interaction'])
    cm = confusion_matrix(true_labels, predicted_labels)

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)

# Main function
def main():
    # Set device to GPU if available, else use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<pad>')

    # Load your trained model
    model_path = "/content/drive/My Drive/Colab Notebooks/Lastmodel2.pth"
    model = load_model(model_path)
    model.to(device)  # Move model to the same device as input data

    # Load datasets for evaluation
    scenario1_test_url = "https://drive.google.com/uc?id=1xwTJQ-4Sx50_v_xQgU-pHszPL_UleBGy"
    scenario2_test_url = "https://drive.google.com/uc?id=1cj_6XMpfO7Fx0DvU-DDonUl1Sdj5Fekg"

    scenario1_test_data = load_dataset_from_url(scenario1_test_url)
    scenario2_test_data = load_dataset_from_url(scenario2_test_url)

    # Evaluate the model for scenario 1
    print("Evaluation for Scenario 1:")
    evaluate_model(model, tokenizer, scenario1_test_data, scenario=1)

    # Evaluate the model for scenario 2
    print("\nEvaluation for Scenario 2:")
    evaluate_model(model, tokenizer, scenario2_test_data, scenario=2)

if __name__ == "__main__":
    main()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation for Scenario 1:
Accuracy: 0.5355355355355356
Confusion Matrix:
[[529  10]
 [454   6]]
Classification Report:
                precision    recall  f1-score   support

No Interaction       0.54      0.98      0.70       539
   Interaction       0.38      0.01      0.03       460

      accuracy                           0.54       999
     macro avg       0.46      0.50      0.36       999
  weighted avg       0.46      0.54      0.39       999


Evaluation for Scenario 2:
Accuracy: 0.5415415415415415
Confusion Matrix:
[[534   5]
 [453   7]]
Classification Report:
                precision    recall  f1-score   support

No Interaction       0.54      0.99      0.70       539
   Interaction       0.58      0.02      0.03       460

      accuracy                           0.54       999
     macro avg       0.56      0.50      0.36       999
  weighted avg       0.56      0.54      0.39       999



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import requests
import io

# Placeholder function for loading your trained model
def load_model(model_path, tokenizer):
    # Load the pretrained GPT2 model for sequence classification
    model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
    model.resize_token_embeddings(len(tokenizer))  # Resize token embeddings to match the tokenizer's vocabulary size

    # Set the pad token ID
    model.config.pad_token_id = tokenizer.pad_token_id

    model.to(device)  # Move the model to the appropriate device

    return model

# Function to load dataset from URL
def load_dataset_from_url(url):
    response = requests.get(url)
    df = pd.read_csv(io.StringIO(response.text))
    return df

# Function to evaluate the model
def evaluate_model(model, tokenizer, test_data, scenario):
    # Tokenize the input text
    if scenario == 1:
        test_encodings = tokenizer(test_data['Drug_Name1'].tolist(), test_data['Drug_Name2'].tolist(), truncation=True, padding=True)
    elif scenario == 2:
        test_encodings = tokenizer(test_data['Patient_Name'].tolist(), test_data['Drug_Name'].tolist(), truncation=True, padding=True)

    # Create PyTorch dataset for test data
    test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                                 torch.tensor(test_encodings['attention_mask']),
                                 torch.tensor(test_data['DDI_Label'].tolist()))

    # Define data loader for test data
    batch_size = 16  # adjust batch size as needed
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    # Set model to evaluation mode
    model.eval()

    # Lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over test data batches
    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Get predicted labels
        predicted_class = torch.argmax(logits, dim=-1)

        # Append true and predicted labels
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predicted_class.tolist())

    # Print true labels, predicted labels, and labels from the dataset
    print("True Labels:", true_labels)
    print("Predicted Labels:", predicted_labels)
    print("Labels from Dataset:", test_data['DDI_Label'].tolist())

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, target_names=['No Interaction', 'Interaction'])
    cm = confusion_matrix(true_labels, predicted_labels)

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)

# Main function
def main():
    # Load the tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<pad>')

    # Specify the path to your trained model
    model_path = "/content/drive/My Drive/Colab Notebooks/Lastmodel2.pth"

    # Load your trained model
    model = load_model(model_path, tokenizer)

    # Load datasets for evaluation
    scenario1_test_url = "https://drive.google.com/uc?id=1xwTJQ-4Sx50_v_xQgU-pHszPL_UleBGy"
    scenario2_test_url = "https://drive.google.com/uc?id=1cj_6XMpfO7Fx0DvU-DDonUl1Sdj5Fekg"

    scenario1_test_data = load_dataset_from_url(scenario1_test_url)
    scenario2_test_data = load_dataset_from_url(scenario2_test_url)

    print("Evaluation for Scenario 1:")
    evaluate_model(model, tokenizer, scenario1_test_data, scenario=1)

    print("\nEvaluation for Scenario 2:")
    evaluate_model(model, tokenizer, scenario2_test_data, scenario=2)

if __name__ == "__main__":
    main()



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation for Scenario 1:
True Labels: [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,

In [ ]:
!pip install flask_bootstrap

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def plot_evaluation_results(true_labels, predicted_labels, labels):
    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)

    # Calculate confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Plot confusion matrix
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')

    # Plot evaluation metrics
    plt.subplot(1, 2, 2)
    metrics = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}
    sns.barplot(x=list(metrics.values()), y=list(metrics.keys()), palette="viridis")
    plt.xlim(0, 1)  # Limit the x-axis from 0 to 1
    plt.xlabel('Score')
    plt.title('Evaluation Metrics')

    plt.tight_layout()
    plt.show()


labels = ['No Interaction', 'Interaction']  # Assuming these are your class labels

# Plot the evaluation results
plot_evaluation_results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask_bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460120 sha256=6d6068ca4b5d71ee380267078150ab13debf859d09ff084127d2628cc3462700
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3928 sha256=7a0ee3eeab7bd134586d2b55595949d24ee0439dd5228007153f8b38f09f2ddc
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask_bootstrap visitor


<function __main__.plot_evaluation_results(true_labels, predicted_labels, labels)>

### ***GUI model ***

In [ ]:
import sys
import requests
!pip install PyQt5
!install cryptography
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QLineEdit, QMessageBox
from cryptography.fernet import Fernet  # for RSA encryption/decryption

install: missing destination file operand after 'cryptography'
Try 'install --help' for more information.
